### Preprocess

1. Load the data from mmadata.zip.
2. Extract the statement and name.
3. Add the prompts to be used with the gpt-4 api.
4. Save the outputs to the /data/modified folder. 

In [1]:
import pandas as pd
import os
import zipfile
import json

In [6]:
# modify the files to change the inputs as expected. 
# This is not deep, but we choose to do it on the system. 
# Negligible lag on my laptop. 
with zipfile.ZipFile("data.zip") as zip_dir:
    files = [file for file in zip_dir.namelist() if 'jsonl' in file and not "MACOS" in file]
    for file in files:
        zip_dir.extract(file)

if not os.path.isdir("data"):
    os.makedirs("data")

if not os.path.isdir("data/modified_mma/"):
    os.makedirs("data/modified_mma/")

for file in files:
    df = pd.read_json(file, lines=True)
    df["input"] = df["input"].apply(lambda x: x.split("\n")[1])
    df["theorem_name"] = df["output"].apply(lambda x: ' '.join(x.split(" ")[0:2]))
    df["theorem_statement"] = df["output"].apply(lambda x: ' '.join(x.split(" ")[2:]))
    df["language"] = file.split("/")[1].split("_")[0].capitalize()
    df["prompt_name"] = df.apply(lambda row: f"{row.language} {row.theorem_name} Translate the theorem name in {row.language} to natural language.", axis = 1)
    df["prompt_statement"] = df.apply(lambda row: f"{row.language} {row.theorem_statement} Translate the theorem statement in {row.language} to natural language.", axis = 1)
    df["prompt_both"] = df.apply(lambda row: f"{row.language} {row.output} Translate the theorem in {row.language} to natural language.", axis = 1)
    #print(df.iloc[0]["language"])
    old_shape = df.shape[0]
    df = df[df["output"] == df["theorem_name"] + " " + df["theorem_statement"]]
    print(f"{file} has been refined to {df.shape[0]} records from {old_shape}")
    df.to_json("data/modified_mma/"+file[4:], orient="records",lines=True)
    os.remove(file)
    

data/lean_test.jsonl has been refined to 1927 records from 1927
data/isabelle_train.jsonl has been refined to 243173 records from 243214
data/isabelle_val.jsonl has been refined to 1023 records from 1024
data/lean_val.jsonl has been refined to 1930 records from 1930
data/lean_train.jsonl has been refined to 84679 records from 84679


We removed some data because it did not contain statements and was hence unsuitable for our experiment. As the numbers above show, we only removed 42 records this way, all from Isabelle. 